In [1]:
import notebook_importer

In [2]:
import spdz
import random
import numpy as np
import zmq

importing notebook from spdz.ipynb


In [3]:
# Bob is party 1
party = 1

# Connect to zmq
context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5555")

In [4]:
# Input dataset
X = np.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [1,1,1]
])

# Output dataset
y = np.array([[0,0,1,1]]).T

# Split input into shares 
X_shares = spdz.share_vec(spdz.wrap(X))
X_alice = spdz.get_share(X_shares, 0)
X_bob = spdz.get_share(X_shares, 1)

# Split output into shares
y_shares = spdz.secure(y)
y_alice = spdz.get_share(y_shares, 0)
y_bob = spdz.get_share(y_shares, 1)

In [5]:
# Bob receives initial weights from Alice
syn0_bob = spdz.swap_shares(np.array("OK"), party, socket)

# Bob sends X and y to Alice
print(spdz.swap_shares(X_alice, party, socket))
print(spdz.swap_shares(y_alice, party, socket))

OK
OK


In [6]:
# Train in sync with Alice

#sigmoid = spdz.SigmoidInterpolated10()
sigmoid = spdz.Sigmoid()

network = spdz.TwoLayerNetwork(sigmoid, party, socket)
network.train(X_bob, y_bob, syn0_bob)

network.print_weights()

Layer 0 weights: 
[[ 9.6731805]
 [-0.2077667]
 [-4.6290403]]
